<a href="https://colab.research.google.com/github/Kaiziferr/Deep_Learning_Workshop/blob/master/01_workshop_forecasting_h5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install h5py

In [2]:
from keras.models import Sequential, model_from_json, model_from_yaml
from keras.layers import Dense
import pandas as pd

In [3]:
url = 'https://raw.githubusercontent.com/Kaiziferr/Deep_Learning_Workshop/master/workshop_keras_sklearn_introduction/diabetes.csv'
data = pd.read_csv(url)
data.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [4]:
X = data.drop('Outcome', axis = 1)
y = data['Outcome']

# MODEL

In [5]:
model = Sequential()
model.add(Dense(12, input_dim = 8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#Compile to model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Ajuste
model.fit(X,y,epochs=150, batch_size=10, verbose=0)

score = model.evaluate(X, y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))


accuracy: 76.69%


# Serializado JSON

In [6]:
model_json = model.to_json()

In [7]:
model_json

'{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 8], "dtype": "float32", "sparse": false, "ragged": false, "name": "dense_input"}}, {"class_name": "Dense", "config": {"name": "dense", "trainable": true, "batch_input_shape": [null, 8], "dtype": "float32", "units": 12, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 8, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_re

In [8]:
with open('model.json', 'w') as f:
  f.write(model_json)

# Serializado pesos HDF5

In [9]:
#Guardan los pesos
model.save_weights('model.h5')

In [10]:
#Cargar json y crear el model
json_file = open('model.json', 'r')
load_model_json = json_file.read()
json_file.close()
load_model_json = model_from_json(load_model_json)

In [11]:
load_model_json.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                108       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compilar modelo
load_model_json.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print('modelo cargado')

modelo cargado


In [13]:
score = load_model_json.evaluate(X, y, verbose=0)
print("%s: %.2f%%" % (load_model_json.metrics_names[1], score[1]*100))

accuracy: 35.42%


# Guardar el modelo en YAML

In [15]:
#Serialize model to yaml
model_yaml = model.to_yaml()
with open('model.yaml', 'w') as yaml_file:
  yaml_file.write(model_yaml)

In [16]:
#Load YAML and create model
yaml_file = open('model.yaml','r')
load_model_yaml = yaml_file.read()
yaml_file.close()
load_model_yaml = model_from_yaml(load_model_yaml)

In [17]:
# load weights
load_model_yaml.load_weights('model.h5')

In [18]:
load_model_yaml.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print('Modelo compilado')

Modelo compilado


In [19]:
#Evaluate loaded model on test data
score = load_model_yaml.evaluate(X, y, verbose=0)
print("%s: %.2f%%" % (load_model_yaml.metrics_names[1], score[1]*100))

accuracy: 76.69%


# Forecasting

In [20]:
# Guardar topologias y pesos
model.save('model_estructura.h5')

# Cargar modelo

In [21]:
from keras.models import load_model

In [22]:
model = load_model('model_estructura.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                108       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


In [23]:
score = model.evaluate(X, y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

accuracy: 76.69%
